In [161]:
import pandas as pd
import numpy as np
import time as t
from pathlib import Path
from xlwings import view

In [162]:
time_stamp = t.time()

In [163]:
# Source files
file_path_src_pq            = Path("D:/Analytik/3_ФТТ/", "_src_pq.xlsx")
file_path_src_ftt           = Path("D:/Analytik/3_ФТТ/_исх/2023/2/", "фтт.xlsx")
file_path_src_ttzz_squ      = Path("D:/Analytik/3_ФТТ/_исх/", "ттзз_скю.xlsx")

# Manuals dataframe
shimpments_qnt_df           = pd.read_excel(file_path_src_pq, "Количество отгрузок")
departments_df              = pd.read_excel(file_path_src_pq, "Справочник отделов")

# Source dataframe
src_ftt_df                  = pd.read_excel(file_path_src_ftt, header=[0, 1])
src_ttzz_squ                = pd.read_excel(file_path_src_ttzz_squ, header=[0, 1])
# Amounting month
month                       = t.strftime("01.02.2023")

c:\Users\derachic_ai\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [164]:
# Количество отгрузок
shimpments_qnt_df = shimpments_qnt_df.loc[shimpments_qnt_df["Начало месяца"] == month]
shimpments_qnt_df.columns = ["Отдел", "Начало месяца", "Значение", "Атрибут"]

# Преобразование object в date
shimpments_qnt_df['Начало месяца'] = pd.to_datetime(shimpments_qnt_df['Начало месяца'], format='%d.%m.%Y')
shimpments_qnt_df = shimpments_qnt_df.set_index(["Отдел", "Начало месяца"])
shimpments_qnt_df

,,Значение,Атрибут
Отдел,Начало месяца,,
Главмолоко,2023-02-01,24,Количество отгрузок
ФЕРМА,2023-02-01,24,Количество отгрузок
VIP,2023-02-01,24,Количество отгрузок


In [165]:
# Сводную таблицу в плоскую
id_vars = src_ftt_df.columns[0:5].to_list()
value_vars = src_ftt_df.columns[5:].to_list()
df = src_ftt_df.melt(id_vars=id_vars, value_vars=value_vars, value_name='Значение', var_name='Дата').dropna()

# Преобразование object в date
df['Дата'] = pd.to_datetime(df['Дата'], format='%d.%m.%y')

# Список полей для переименования
columns = []
for x in df.columns[0:5]:
    columns.append(x[1])    # берем заголовки первого уровня
for x in df.columns[5:]:
    columns.append(x)    

df.columns = columns

df = pd.merge(df, departments_df, on=["Отдел продаж", "Подразделение"])
df["Начало месяца"] = pd.to_datetime(df['Дата']).dt.to_period('M').dt.to_timestamp()

In [166]:
# Продажи, руб.
sales = df.groupby(["Отдел", "Начало месяца"]).sum()
sales["Атрибут"] = "Продажи, руб."
sales


,,Значение,Атрибут
Отдел,Начало месяца,,
VIP,2023-02-01,5066844.88,"Продажи, руб."
Главмолоко,2023-02-01,39351827.66,"Продажи, руб."
ФЕРМА,2023-02-01,30028787.53,"Продажи, руб."


In [167]:
# Средняя отгрузка
avg_shimpment_df = pd.merge(sales, shimpments_qnt_df, on=["Отдел", "Начало месяца"])
avg_shimpment_df["Значение"] = avg_shimpment_df["Значение_x"] / avg_shimpment_df["Значение_y"]
avg_shimpment_df.drop(columns=["Значение_x", "Значение_y", "Атрибут_x", "Атрибут_y"], inplace=True)
avg_shimpment_df["Атрибут"] = "Средняя отгрузка"
avg_shimpment_df

,,Значение,Атрибут
Отдел,Начало месяца,,
VIP,2023-02-01,2.111185e+05,Средняя отгрузка
Главмолоко,2023-02-01,1.639659e+06,Средняя отгрузка
ФЕРМА,2023-02-01,1.251199e+06,Средняя отгрузка


In [168]:
# Количество ТРТ
qnt_rtt = df.groupby(["Отдел", "Начало месяца"])["РТТ"].nunique()
qnt_rtt = qnt_rtt.to_frame(name="Значение")
qnt_rtt["Атрибут"] = "Количество ТРТ"
qnt_rtt


,,Значение,Атрибут
Отдел,Начало месяца,,
VIP,2023-02-01,83,Количество ТРТ
Главмолоко,2023-02-01,2466,Количество ТРТ
ФЕРМА,2023-02-01,2415,Количество ТРТ


In [169]:
# Количество должностей
qnt_posts = df.groupby(["Отдел", "Начало месяца"])["Должность ТА"].nunique()
qnt_posts = qnt_posts.to_frame(name="Значение")
qnt_posts["Атрибут"] = "Количество должностей"
qnt_posts

,,Значение,Атрибут
Отдел,Начало месяца,,
VIP,2023-02-01,8,Количество должностей
Главмолоко,2023-02-01,29,Количество должностей
ФЕРМА,2023-02-01,23,Количество должностей


In [170]:
# Количество ТА
qnt_ta = df.groupby(["Отдел", "Начало месяца"])["ФИО ТА"].nunique()
qnt_ta = qnt_ta.to_frame(name="Значение")
qnt_ta["Атрибут"] = "Количество ТА"
qnt_ta

,,Значение,Атрибут
Отдел,Начало месяца,,
VIP,2023-02-01,6,Количество ТА
Главмолоко,2023-02-01,30,Количество ТА
ФЕРМА,2023-02-01,22,Количество ТА


In [171]:
# Количество заказов
qnt_orders = df[["Отдел", "Подразделение", "Начало месяца", "РТТ", "Дата"]].drop_duplicates()
qnt_orders = qnt_orders.groupby(["Отдел", "Начало месяца"])["РТТ"].count()
qnt_orders = qnt_orders.to_frame(name="Значение")
qnt_orders["Атрибут"] = "Количество заказов"
qnt_orders

,,Значение,Атрибут
Отдел,Начало месяца,,
VIP,2023-02-01,1099,Количество заказов
Главмолоко,2023-02-01,15398,Количество заказов
ФЕРМА,2023-02-01,13225,Количество заказов


In [172]:
# Среднее кол-во заказов в ТРТ
avg_orders_qnt = pd.merge(qnt_orders, qnt_rtt, on=["Отдел", "Начало месяца"])
avg_orders_qnt["Значение"] = avg_orders_qnt["Значение_x"]/avg_orders_qnt["Значение_y"]
avg_orders_qnt.drop(columns=["Значение_x", "Атрибут_x", "Значение_y", "Атрибут_y"], inplace=True)
avg_orders_qnt["Атрибут"] = "Среднее кол-во заказов в ТРТ"
avg_orders_qnt

,,Значение,Атрибут
Отдел,Начало месяца,,
VIP,2023-02-01,13.240964,Среднее кол-во заказов в ТРТ
Главмолоко,2023-02-01,6.244120,Среднее кол-во заказов в ТРТ
ФЕРМА,2023-02-01,5.476190,Среднее кол-во заказов в ТРТ


In [173]:
# Среднее кол-во ТРТ в день на должность
avq_rtt_day_post = qnt_orders.merge(shimpments_qnt_df, on=["Отдел", "Начало месяца"]).merge(qnt_posts, on=["Отдел", "Начало месяца"])
avq_rtt_day_post["Значение"] = avq_rtt_day_post["Значение_x"]/avq_rtt_day_post["Значение_y"]/avq_rtt_day_post["Значение"]
avq_rtt_day_post.drop(columns=["Значение_x", "Атрибут_x", "Значение_y", "Атрибут_y"], inplace=True)
avq_rtt_day_post["Атрибут"] = "Среднее кол-во ТРТ в день на должность"
avq_rtt_day_post

,,Значение,Атрибут
Отдел,Начало месяца,,
VIP,2023-02-01,5.723958,Среднее кол-во ТРТ в день на должность
Главмолоко,2023-02-01,22.123563,Среднее кол-во ТРТ в день на должность
ФЕРМА,2023-02-01,23.958333,Среднее кол-во ТРТ в день на должность


In [174]:
# Количество ТРТ с грппировкой по ЗЗ
qnt_rtt_by_orders = df[["Отдел", "Подразделение", "Начало месяца", "РТТ", "Дата"]].drop_duplicates()
qnt_rtt_by_orders = qnt_rtt_by_orders.groupby(["Отдел", "Начало месяца", "РТТ"])["Дата"].count()
qnt_rtt_by_orders = qnt_rtt_by_orders.to_frame(name="Значение")
qnt_rtt_by_orders["Атрибут"] = "Количество ТРТ с грппировкой по ЗЗ"
qnt_rtt_by_orders["Атрибут1"] = np.where(qnt_rtt_by_orders["Значение"] < 3, "1-2 разовых заказов за месяц",
                                  np.where(qnt_rtt_by_orders["Значение"] < 5, "3-4 разовых заказов за месяц",
                                           np.where(qnt_rtt_by_orders["Значение"] < 8, "5-7 разовых заказов за месяц", 
                                                    np.where(qnt_rtt_by_orders["Значение"] >= 8, "8 и выше заказов за месяц", 0)
                                                    )
                                            )
                                  )
qnt_rtt_by_orders = qnt_rtt_by_orders.groupby(["Отдел", "Начало месяца", "Атрибут", "Атрибут1"]).count()
qnt_rtt_by_orders = qnt_rtt_by_orders.reset_index(level=["Атрибут", "Атрибут1"])
qnt_rtt_by_orders

Атрибут  \
Отдел      Начало месяца                                       
VIP        2023-02-01     Количество ТРТ с грппировкой по ЗЗ   
           2023-02-01     Количество ТРТ с грппировкой по ЗЗ   
           2023-02-01     Количество ТРТ с грппировкой по ЗЗ   
           2023-02-01     Количество ТРТ с грппировкой по ЗЗ   
Главмолоко 2023-02-01     Количество ТРТ с грппировкой по ЗЗ   
           2023-02-01     Количество ТРТ с грппировкой по ЗЗ   
           2023-02-01     Количество ТРТ с грппировкой по ЗЗ   
           2023-02-01     Количество ТРТ с грппировкой по ЗЗ   
ФЕРМА      2023-02-01     Количество ТРТ с грппировкой по ЗЗ   
           2023-02-01     Количество ТРТ с грппировкой по ЗЗ   
           2023-02-01     Количество ТРТ с грппировкой по ЗЗ   
           2023-02-01     Количество ТРТ с грппировкой по ЗЗ   

                                              Атрибут1  Значение  
Отдел      Начало месяца                                          
VIP        2023-02-01     1-2 разовых заказов за месяц        19  
           2023-02-01     3-4 разовых заказов за месяц        14  
           2023-02-01     5-7 разовых заказов за месяц        14  
           2023-02-01        8 и выше заказов за месяц        36  
Главмолоко 2023-02-01     1-2 разовых заказов за месяц       426  
           2023-02-01     3-4 разовых заказов за месяц       630  
           2023-02-01     5-7 разовых заказов за месяц       624  
           2023-02-01        8 и выше заказов за месяц       786  
ФЕРМА      2023-02-01     1-2 разовых заказов за месяц       583  
           2023-02-01     3-4 разовых заказов за месяц       676  
           2023-02-01     5-7 разовых заказов за месяц       537  
           2023-02-01        8 и выше заказов за месяц       619

In [175]:
# Средний заказ, руб.
avg_order = sales.merge(qnt_orders, on=["Отдел", "Начало месяца"])
avg_order["Значение"] = avg_order["Значение_x"]/avg_order["Значение_y"]
avg_order.drop(columns=["Значение_x", "Атрибут_x", "Значение_y", "Атрибут_y"], inplace=True)
avg_order["Атрибут"] = "Средний заказ, руб."
avg_order

,,Значение,Атрибут
Отдел,Начало месяца,,
VIP,2023-02-01,4610.413904,"Средний заказ, руб."
Главмолоко,2023-02-01,2555.645386,"Средний заказ, руб."
ФЕРМА,2023-02-01,2270.607753,"Средний заказ, руб."


In [176]:
# Средняя продажа в ТРТ, руб
avg_rtt_sale = sales.merge(qnt_rtt, on=["Отдел", "Начало месяца"])
avg_rtt_sale["Значение"] = avg_rtt_sale["Значение_x"]/avg_rtt_sale["Значение_y"]
avg_rtt_sale.drop(columns=["Значение_x", "Атрибут_x", "Значение_y", "Атрибут_y"], inplace=True)
avg_rtt_sale["Атрибут"] = "Средняя продажа в ТРТ, руб"
avg_rtt_sale

,,Значение,Атрибут
Отдел,Начало месяца,,
VIP,2023-02-01,61046.323855,"Средняя продажа в ТРТ, руб"
Главмолоко,2023-02-01,15957.756553,"Средняя продажа в ТРТ, руб"
ФЕРМА,2023-02-01,12434.280551,"Средняя продажа в ТРТ, руб"


In [177]:
# Активный ассортимент
id_vars = src_ttzz_squ.columns[:7].to_list()
value_vars = src_ttzz_squ.columns[7:].to_list()
df2 = src_ttzz_squ.melt(id_vars=id_vars, value_vars=value_vars, var_name=["Начало месяца", "Параметр"]).dropna()

# Преобразование object в date
df2['Начало месяца'] = pd.to_datetime(df2["Начало месяца"], format='%Y.%m')

# Список полей для переименования
columns = []
for x in df2.columns[0:7]:
    columns.append(x[1])    # берем заголовки первого уровня
for x in df2.columns[7:]:
    columns.append(x)    

df2.columns = columns
df2 = pd.merge(df2, departments_df, on=["Отдел продаж", "Подразделение"])

active_assort = df2.groupby(["Отдел", "Начало месяца", "Торговая марка"])["ТМЦ"].nunique()
active_assort = active_assort.to_frame("Значение")
active_assort["Атрибут"] = "Активный ассортимент"
active_assort = active_assort.rename_axis(index={"Торговая марка" : "Атрибут1"})
active_assort = active_assort.reset_index(level= "Атрибут1")
active_assort = active_assort[["Значение", "Атрибут", "Атрибут1"]]
active_assort

Значение               Атрибут              Атрибут1
Отдел Начало месяца                                                      
VIP   2022-11-01            4  Активный ассортимент                Без ТМ
      2022-11-01           30  Активный ассортимент             Белавушка
      2022-11-01            4  Активный ассортимент  Белорусский Стандарт
      2022-11-01           17  Активный ассортимент            Главмолоко
      2022-11-01            4  Активный ассортимент           Доброе утро
...                       ...                   ...                   ...
ФЕРМА 2023-02-01            4  Активный ассортимент  Белорусский Стандарт
      2023-02-01           10  Активный ассортимент            Главмолоко
      2023-02-01           15  Активный ассортимент                 Живая
      2023-02-01            6  Активный ассортимент                 Ретро
      2023-02-01           32  Активный ассортимент                 ФЕРМА

[82 rows x 3 columns]

In [178]:
ftt_df = pd.concat([active_assort
                    , avg_rtt_sale
                    , avg_order
                    , qnt_rtt_by_orders
                    , avq_rtt_day_post
                    , avg_orders_qnt
                    , qnt_orders
                    , qnt_ta
                    , qnt_posts
                    , qnt_rtt
                    , avg_shimpment_df
                    , sales
                    , shimpments_qnt_df
                    ])
view(ftt_df)

In [179]:
t.time() - time_stamp 

4.886569499969482